In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
import pdb
import _pickle as cPickle
import time
import subprocess
import glob

import random
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict, Counter
from ncrp import Topic, Doc, init, sample, get_perplexity, get_topic_specialization, get_hierarchical_affinities, get_freq_tokens_ncrp, get_docs
from configure import get_config

# load config & data 

In [3]:
config = get_config(nb_name)
np.random.seed(config.seed)
random.seed(config.seed)

In [4]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))

Exception ignored in: <_io.FileIO name='data/bags/instances.pkl' mode='rb' closefd=True>


In [5]:
config.n_doc = len(instances_train)
config.n_vocab = len(bow_idxs)
config.n_doc, config.n_vocab

(31943, 1035)

# run

## initialize log

In [6]:
checkpoint = []
ppl_min = np.inf
epoch = 0

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','VALID:','TEST:','SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','PPL','PPL', 'PPL','1', '2', '3', 'CHILD', 'OTHER']]))))

def update_checkpoint(config, checkpoint, epoch):
    checkpoint.append(config.path_model + '-%i' % epoch)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0)
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))

## initialize data

In [7]:
topic_root = Topic(idx='0', sibling_idx=0, parent=None, depth=0, config=config)
train_docs = get_docs(instances_train, config)
dev_docs = get_docs(instances_dev, config)
test_docs = get_docs(instances_test, config)
init(train_docs, dev_docs, test_docs, topic_root)

0 10000 20000 30000 0 0 

## run

In [8]:
time_start = time.time()
while epoch < config.n_epochs:
    sample(train_docs, dev_docs, test_docs, topic_root)
    ppl_train = get_perplexity(train_docs, topic_root)
    ppl_dev = get_perplexity(dev_docs, topic_root)
    
    if ppl_dev < ppl_min:
        ppl_min = ppl_dev
        ppl_test = get_perplexity(test_docs, topic_root)
        cPickle.dump([test_docs, topic_root], open(config.path_model + '-%i'%epoch, 'wb'))
        update_checkpoint(config, checkpoint, epoch)
        
    depth_spec = get_topic_specialization(test_docs, topic_root)
    hierarchical_affinities = get_hierarchical_affinities(topic_root)
    
    clear_output()
    time_log = int(time.time() - time_start)
    time_start = time.time()
    log_series = pd.Series([time_log, epoch, 0, \
            '%.0f'%ppl_train, '%.0f'%ppl_dev, '%.0f'%ppl_test, \
            '%.2f'%depth_spec[1], '%.2f'%depth_spec[2], '%.2f'%depth_spec[3], \
            '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
            index=log_df.columns)
    log_df.loc[epoch] = log_series    
    display(log_df)
    get_freq_tokens_ncrp(topic_root, idx_to_word, bow_idxs)
    
    cPickle.dump(log_df, open(config.path_log, 'wb'))
    epoch += 1

TRAIN: VALID: TEST: SPEC:             HIER:      
   Time  Ep Ct    PPL    PPL   PPL     1     2     3 CHILD OTHER
0   140   0  0    495    472   465  0.05  0.37  0.49  0.80  0.74
1   148   1  0    487    467   457  0.05  0.43  0.51  0.73  0.65
2   149   2  0    474    457   448  0.05  0.45  0.53  0.67  0.58
3   152   3  0    453    436   426  0.05  0.47  0.54  0.61  0.53
4   157   4  0    428    414   406  0.05  0.47  0.58  0.58  0.45
5   166   5  0    404    393   382  0.06  0.47  0.59  0.67  0.52
6   172   6  0    388    381   371  0.06  0.42  0.58  0.57  0.42
7   173   7  0    374    371   361  0.06  0.46  0.60  0.51  0.35
8   192   8  0    363    361   351  0.07  0.49  0.61  0.61  0.43
9   184   9  0    351    351   340  0.07  0.51  0.62  0.54  0.38
10  187  10  0    348    347   334  0.08  0.48  0.61  0.49  0.35
11  179  11  0    349    347   340  0.08  0.46  0.62  0.45  0.30
12  194  12  0    345    340   334  0.08  0.49  0.61  0.53  0.38
13  197  13  0    340    336   331  0.08  0.50  0.62  0.45  0.34
14  208  14  0    341    335   332  0.09  0.50  0.60  0.44  0.33
15  225  15  0    336    331   325  0.09  0.52  0.59  0.51  0.41
16  227  16  0    329    325   321  0.09  0.53  0.60  0.44  0.36
17  228  17  0    329    333   321  0.09  0.52  0.58  0.43  0.35
18  225  18  0    332    333   321  0.09  0.51  0.57  0.42  0.34
19  235  19  0    330    329   321  0.09  0.51  0.58  0.49  0.41
20  252  20  0    326    325   321  0.09  0.51  0.57  0.45  0.38
21  247  21  0    328    327   321  0.09  0.52  0.58  0.41  0.34
22  259  22  0    322    320   317  0.09  0.55  0.58  0.48  0.39
23  275  23  0    316    317   311  0.10  0.56  0.58  0.52  0.42
24  269  24  0    319    319   311  0.10  0.55  0.57  0.51  0.41
25  271  25  0    320    322   311  0.10  0.56  0.56  0.49  0.41
26  276  26  0    319    319   311  0.10  0.55  0.56  0.47  0.38
27  273  27  0    322    324   311  0.10  0.55  0.56  0.43  0.36
28  274  28  0    325    326   311  0.10  0.54  0.56  0.42  0.35
29  275  29  0    323    321   311  0.10  0.54  0.56  0.41  0.35

 0 31943 292385.0 ! nice bought price quality love perfect recommend 'm made
   0-1 3732 28689.0 ! cover color love keyboard mac pro apple perfectly easy
     0-1-2 3732 2770.0 put fingerprints ports happy im recommend mail mac cover bottom
   0-2 2684 21187.0 carry travel handle straps back work pack pockets compartments shoulder
     0-2-1 2684 3768.0 camera lenses lens canon equipment tripod flash body accessories batteries
   0-3 2538 21733.0 pocket power netbook sleeve mouse cord room charger ipad carry
     0-3-1 2538 2113.0 player cords handy charger zips airport hold accessory extra drive
   0-4 2200 17365.0 & ; sleeve pro inch protection air snug zipper retina
     0-4-2 2200 1720.0 inches loose zipper thick foam layer vaio retina tight major
   0-5 2387 19347.0 zipper strap months broke handle years straps year shoulder started
     0-5-2 2387 2233.0 year main loved binder luggage cheaply lasted swissgear abuse lenses
   0-6 2082 19050.0 cover bottom top keyboard plastic part

Exception ignored in: <_io.FileIO name='model/bags/ncrp/bags-mncrp-alp1051-eta5-gam0.01-epoch30/log' mode='wb' closefd=True>
